# Observaciones convencionales horarias

Se extraen los datos climáticos horarios de las estaciones automáticas de la AEMET generados en las 24 h últimas. Se pueden descargar todas las estaciones o de las estaciones determinadas.

Si se hiciera recurrentemente se 

In [1]:
import os
base_path = os.getcwd().replace('\\', '/') +'/'

In [150]:
# cargar funciones de la API de la AEMET
%run funciones_opendata_AEMET.ipynb

In [2]:
import requests

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import pandas as pd
import numpy as np
import datetime
import time

from pathlib import Path

In [3]:
# Archivo donde está guardada la API key
API_path = base_path + 'apikey_AEMET.txt'
# Carpeta en la que se guardarán las series
series_path = 'F:/Series/AEMET/Clima/Horario/'
series_path = base_path + 'Horario/'
if os.path.exists(series_path) is False:
    os.makedirs(series_path)
# Estaciones a importar: todas o una lista de las que se quiera
codigos = ['todas']
# Archivo con los datos de las estaciones automáticas de la AEMET
ruta_estaciones = base_path + 'Estaciones_Automaticas.csv'

## 1. Estaciones automáticas

Se importa el listado de las estaciones automáticas de la AEMET en toda España. Estas estaciones son las susceptibles de aportar datos por este medio.

In [111]:
estaciones = pd.read_csv(ruta_estaciones)
estaciones.drop(['FID'], axis=1, inplace=True)
estaciones.columns = ['idema', 'ubi', 'PROVINCIA', 'alt', 'COORD_X', 'COORD_Y', 'VAR_OBSERV', 'DATUM', 'TIPO']
estaciones.sort_values('idema', inplace=True)
estaciones.set_index('idema', drop=False, inplace=True)

# Corregir caracteres conflictivos en el nombre de las estaciones
for i in estaciones.index:
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('/', '-')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('"', '')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Á', 'A')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('É', 'E')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Í', 'I')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Ó', 'O')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Ú', 'U')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('À', 'A')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('È', 'E')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Ç', 'C')
    estaciones.loc[i, 'ubi'] = estaciones.loc[i, 'ubi'].replace('Ñ', 'N')

print('Nº de estaciones automáticas de la AEMET: ', estaciones.shape[0])

Nº de estaciones automáticas de la AEMET:  852


In [114]:
estaciones.head(2)

,idema,ubi,PROVINCIA,alt,COORD_X,COORD_Y,VAR_OBSERV,DATUM,TIPO
idema,,,,,,,,,
0002I,0002I,VANDELLOS,TARRAGONA,32.0,825831.0,4541326.0,hreNvIHBVmtTMpP,ETRS89,AUTOMATICAS
0009X,0009X,ALFORJA,TARRAGONA,406.0,832278.0,4570083.0,hHVmtTMpP,ETRS89,AUTOMATICAS


## 1. Extraer todas las estaciones

Se extraen los datos climáticos horarios disponibles de las últimas 24 h.


### 1.2 Extraer datos

In [151]:
extract_hourly_all(API_path, series_path)

{'descripcion': 'exito', 'estado': 200, 'datos': 'https://opendata.aemet.es/opendata/sh/9c512188', 'metadatos': 'https://opendata.aemet.es/opendata/sh/55c2971b'}


## 2. Extraer una única estación

### 2.1 Funciones

### 2.2 Aplicarlo a todas las estaciones en bucle

In [ ]:
if codigos[0] != 'todas':
    for stn in codigos:
        # Nombre de la estación
        nombre = estaciones.loc[stn, ubi]
        # Extraer datos
        data_stn = extract_data_stn(stn, nombre, API_path)
        # Generar la serie
        generate_serie_stn(stn, nombre, data_stn, series_path)

In [ ]:
import schedule